In [1]:
import numpy as np
import pandas as pd
import stan

from datamodel import SPX, StockData
from model_testing import Model, WalkForwardAnalysis, create_lags

# Workaround for jupyter notebook event loop. Needed for pystan
import nest_asyncio
nest_asyncio.apply()

In [2]:
spx = SPX()
data = StockData(["^VIX"])

spx_wk_prices = spx.prices.resample("W-FRI").last()
spx_wk_returns = (np.log(spx_wk_prices) - np.log(spx_wk_prices.shift(1))).dropna()
spx_wk_vol = np.sqrt(spx.vol.resample("W-FRI").sum())

In [3]:
spx_vol_lags = create_lags(np.log(np.sqrt(spx.vol)), 21, "spx_vol")
vix_lags = create_lags(np.log(data.prices.VIX["close"]), 21, "vix")
spx_return_lags = create_lags(spx.returns, 21, "spx_returns")

indep_var = pd.concat([spx_vol_lags, vix_lags, spx_return_lags], axis=1).dropna()
dep_var = np.log(spx_wk_vol.shift(-1).dropna())

common_index = indep_var.index.intersection(dep_var.index)

indep_var = indep_var.loc[common_index]
dep_var = dep_var.loc[common_index]

In [7]:
model_spec = """
data {
    int N;  // Number of observations
    int p;  // Number of variables
    vector[N] y;
    matrix[N, p] X;
    
    int N_tilde;  // Number of observations in out-of-sample data
    vector[N_tilde] x_tilde; 
}
parameters {
    real mu;
    real<lower=0> rho;
    vector[p] beta;
    real<lower=0> sigma;
}
model {
    mu ~ normal(0, 10);
    rho ~ normal(0, 1);
    beta ~ normal(0, rho);
    sigma ~ normal(0, 10);
    
    y ~ normal(mu + (X * beta), sigma);
}
generated quantities {
    vector[N_tilde] y_tilde = normal_rng(mu + (X * beta), sigma);
}
"""

In [8]:
model_spec = """
data {
    int N;  // Number of observations
    int p;  // Number of variables
    vector[N] y;
    matrix[N, p] X;
}
parameters {
    real mu;
    real<lower=0> rho;
    vector[p] beta;
    real<lower=0> sigma;
}
model {
    mu ~ normal(0, 10);
    rho ~ normal(0, 1);
    beta ~ normal(0, rho);
    sigma ~ normal(0, 10);
    
    y ~ normal(mu + (X * beta), sigma);
}
"""

In [9]:
data = {"N": len(indep_var),
          "p": len(indep_var.columns),
          "y": dep_var.values,
          "X": indep_var.values}
model = stan.build(model_spec, data=data)

Building: 0.1s
Building: 0.2s
Building: 0.3s
Building: 0.4s
Building: 0.6s
Building: 0.7s
Building: 0.8s
Building: 0.9s
Building: 1.0s
Building: 1.1s
Building: 1.2s
Building: 1.3s
Building: 1.4s
Building: 1.5s
Building: 1.6s
Building: 1.7s
Building: 1.8s
Building: 1.9s
Building: 2.0s
Building: 2.1s
Building: 2.2s
Building: 2.4s
Building: 2.5s
Building: 2.6s
Building: 2.7s
Building: 2.8s
Building: 2.9s
Building: 3.0s
Building: 3.1s
Building: 3.2s
Building: 3.3s
Building: 3.4s
Building: 3.5s
Building: 3.6s
Building: 3.7s
Building: 3.8s
Building: 3.9s
Building: 4.0s
Building: 4.1s
Building: 4.2s
Building: 4.3s
Building: 4.4s
Building: 4.5s
Building: 4.7s
Building: 4.8s
Building: 4.9s
Building: 5.0s
Building: 5.1s
Building: 5.2s
Building: 5.3s
Building: 5.4s
Building: 5.5s
Building: 5.6s
Building: 5.7s
Building: 5.8s
Building: 5.9s
Building: 6.0s
Building: 6.1s
Building: 6.2s
Building: 6.3s
Building: 6.4s


In file included from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math/prim/fun.hpp:110,
                 from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math/rev/fun.hpp:7,
                 from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math/rev.hpp:10,
                 from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math.hpp:19,
                 from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/model/model_header.hpp:4,
                 from /home/eadains/.cache/httpstan/4.5.0/models/s5plwmfn/model_s5plwmfn.cpp:2:
/home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math/prim/fun/generalized_inverse.hpp: In function ‘Eigen::Matrix<typename stan::value_type<T>::type, EigMat::ColsAtCompileTime, EigMat::RowsAtCompileTime> stan::math::gener

Building: 6.5s
Building: 6.6s
Building: 6.7s


In file included from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math/prim/fun.hpp:321,
                 from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math/rev/fun.hpp:7,
                 from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math/rev.hpp:10,
                 from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math.hpp:19,
                 from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/model/model_header.hpp:4,
                 from /home/eadains/.cache/httpstan/4.5.0/models/s5plwmfn/model_s5plwmfn.cpp:2:
/home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math/prim/fun/tail.hpp: In function ‘std::vector<T1> stan::math::tail(const std::vector<T1>&, size_t)’:
/home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/http

Building: 6.8s
Building: 7.0s
Building: 7.1s
Building: 7.2s
Building: 7.3s
Building: 7.4s
Building: 7.5s
Building: 7.6s
Building: 7.7s
Building: 7.8s
Building: 7.9s
Building: 8.0s
Building: 8.1s


...

In file included from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/model/indexing.hpp:7,
                 from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/model/model_header.hpp:17,
                 from /home/eadains/.cache/httpstan/4.5.0/models/s5plwmfn/model_s5plwmfn.cpp:2:
/home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/model/indexing/assign_varmat.hpp: In function ‘void stan::model::assign(Mat1&&, const Mat2&, const char*, const stan::model::index_multi&, const stan::model::index_multi&)’:
/home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/model/indexing/assign_varmat.hpp:329:9: warning: typedef ‘using pair_type = struct std::pair<int, std::vector<int, stan::math::arena_allocator<int> > >’ locally defined but not used [-Wunused-local-typedefs]
  329 |   using pair_type = std::pair<int, arena_vec>;
      |         ^~~~~~~~~


Building: 11.6s
Building: 11.7s
Building: 11.8s
Building: 11.9s
Building: 12.0s
Building: 12.1s
Building: 12.2s
Building: 12.3s
Building: 12.4s
Building: 12.5s
Building: 12.6s
Building: 12.7s
Building: 12.8s
Building: 12.9s
Building: 13.0s
Building: 13.1s
Building: 13.2s
Building: 13.3s
Building: 13.4s
Building: 13.5s
Building: 13.7s
Building: 13.8s
Building: 13.9s
Building: 14.0s
Building: 14.1s
Building: 14.2s
Building: 14.3s
Building: 14.4s
Building: 14.5s
Building: 14.6s
Building: 14.7s
Building: 14.8s
Building: 14.9s
Building: 15.0s
Building: 15.1s
Building: 15.2s
Building: 15.3s
Building: 15.4s
Building: 15.5s
Building: 15.6s
Building: 15.7s
Building: 15.8s
Building: 16.0s
Building: 16.1s
Building: 16.2s
Building: 16.3s
Building: 16.4s
Building: 16.5s


In file included from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math/prim/err/check_not_nan.hpp:5,
                 from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math/prim/err/check_2F1_converges.hpp:5,
                 from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math/prim/err.hpp:4,
                 from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math/rev/core/profiling.hpp:9,
                 from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math/rev/core.hpp:53,
                 from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math/rev.hpp:8,
                 from /home/eadains/VolatilityTrading/venv/lib/python3.8/site-packages/httpstan/include/stan/math.hpp:19,
                 from /home/eadains/VolatilityTrading/venv

Building: 16.6s
Building: 16.7s
Building: 16.8s
Building: 16.9s
Building: 17.0s
Building: 17.1s
Building: 17.2s
Building: 17.3s
Building: 17.4s
Building: 17.5s
Building: 17.6s
Building: 17.7s
Building: 17.8s
Building: 17.9s
Building: 18.0s
Building: 18.2s
Building: 18.3s
Building: 18.4s
Building: 18.5s
Building: 18.6s
Building: 18.7s
Building: 18.8s
Building: 18.9s
Building: 19.0s
Building: 19.1s
Building: 19.2s
Building: 19.3s
Building: 19.4s
Building: 19.5s
Building: 19.6s
Building: 19.7s
Building: 19.8s
Building: 19.9s
Building: 20.0s
Building: 20.1s
Building: 20.2s
Building: 20.3s
Building: 20.5s
Building: 20.6s
Building: 20.7s
Building: 20.8s
Building: 20.9s
Building: 21.0s
Building: 21.1s
Building: 21.2s
Building: 21.3s
Building: 21.4s
Building: 21.5s
Building: 21.6s
Building: 21.7s
Building: 21.8s
Building: 21.9s
Building: 22.0s
Building: 22.1s
Building: 22.2s
Building: 22.3s
Building: 22.4s
Building: 22.6s
Building: 22.7s
Building: 22.8s
Building: 22.9s
Building: 23.0s
Building

Building: 30.4s, done.


In [10]:
sample = model.sample(num_chains=4, num_samples=2500)

Sampling:   0%
Sampling:   0% (1/14000)
Sampling:   0% (2/14000)
Sampling:   0% (3/14000)
Sampling:   0% (4/14000)
Sampling:   1% (103/14000)
Sampling:   1% (202/14000)
Sampling:   2% (301/14000)
Sampling:   3% (400/14000)
Sampling:   4% (500/14000)
Sampling:   4% (600/14000)
Sampling:   5% (700/14000)
Sampling:   6% (800/14000)
Sampling:   6% (900/14000)
Sampling:   7% (1000/14000)
Sampling:   8% (1100/14000)
Sampling:   9% (1200/14000)
Sampling:   9% (1300/14000)
Sampling:  10% (1400/14000)
Sampling:  11% (1500/14000)
Sampling:  11% (1600/14000)
Sampling:  12% (1700/14000)
Sampling:  13% (1800/14000)
Sampling:  14% (1900/14000)
Sampling:  14% (2000/14000)
Sampling:  15% (2100/14000)
Sampling:  16% (2200/14000)
Sampling:  16% (2300/14000)
Sampling:  17% (2400/14000)
Sampling:  18% (2500/14000)
Sampling:  19% (2600/14000)
Sampling:  19% (2700/14000)
Sampling:  20% (2800/14000)
Sampling:  21% (2900/14000)
Sampling:  21% (3000/14000)
Sampling:  22% (3100/14000)
Sampling:  23% (3200/14000

In [20]:
sample["summary"]

AssertionError: summary

In [27]:
sample.to_frame()

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,mu,rho,beta.1,...,beta.58,beta.59,beta.60,beta.61,beta.62,beta.63,beta.64,beta.65,beta.66,sigma
draws,,,,,,,,,,,,,,,,,,,,,
0,882.434780,0.999162,0.004109,10.0,1023.0,0.0,-849.671383,-4.546970,0.160809,0.315146,...,-0.071876,-0.092419,0.017753,0.221412,-0.147647,-0.131911,-0.131909,0.098026,-0.100839,0.201975
1,880.122770,0.921791,0.004177,10.0,1023.0,0.0,-843.452550,-5.125916,0.166193,0.297089,...,-0.045635,-0.032164,-0.017813,-0.098653,-0.180772,0.120176,0.079124,0.201756,-0.002970,0.196395
2,873.317333,0.972938,0.004338,10.0,1023.0,0.0,-835.610403,-4.236260,0.242147,0.305447,...,0.265354,-0.272796,-0.118442,0.001361,0.085410,0.200944,0.190556,0.176219,0.138382,0.201043
3,890.403531,0.999402,0.004319,10.0,1023.0,0.0,-859.417519,-5.095953,0.152165,0.303801,...,-0.265937,0.078193,-0.219276,-0.298835,-0.172210,-0.184456,0.082355,0.181228,0.043764,0.196215
4,895.029084,0.978569,0.004109,10.0,1023.0,0.0,-859.403017,-4.509452,0.187624,0.312779,...,-0.083663,-0.067486,-0.123326,-0.036415,0.030318,0.215447,-0.112425,-0.025585,0.036962,0.205897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,880.771018,0.999652,0.004319,10.0,1023.0,0.0,-840.401378,-4.200955,0.195423,0.293191,...,-0.014657,-0.029862,-0.356982,-0.069147,-0.178322,0.311311,-0.279923,-0.068978,-0.409477,0.192560
9996,883.759529,0.988464,0.004109,10.0,1023.0,0.0,-851.591614,-5.116960,0.186060,0.251964,...,-0.157222,0.007443,0.022032,-0.035243,-0.145685,-0.425074,-0.034052,0.025781,-0.138155,0.195517
9997,881.632543,0.998449,0.004177,10.0,1023.0,0.0,-840.008918,-4.678708,0.197163,0.301283,...,0.149534,0.009622,0.296438,-0.056041,0.178218,-0.230092,0.044792,0.263724,0.085531,0.186853
